In [1]:
# 10/19/2021
# No need to run first section again. 
# Run import statements then proceed to Preprocess Data section where csv is loaded in.

# BINNING NEEDS WORK IN ORDER FOR MODELS TO RUN

In [2]:
# import dependencies
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
from config import Password_2, Username_2, Host_2,DB2
import pandas as pd
import pickle

In [3]:
# connect to database
engine = create_engine(f'postgresql://{Username_2}:{Password_2}@{Host_2}:5432/{DB2}', echo=False)
connection = engine.connect()
inspector = inspect(engine)
inspector.get_table_names()

['Movie_Genre_Junction',
 'Role',
 'Movie_Person_Role_Junction',
 'Genre',
 'Movie',
 'Person']

In [4]:
# fetch data from database
cursor = engine.execute('SELECT public."Movie"."movie_id", public."Movie"."movie_rating", public."Movie"."release_date", public."Movie"."runtime", public."Movie"."metascore", \
    public."Movie"."imdb_rating", public."Movie"."boxoffice", public."Movie"."budget", public."Movie"."buzz"  \
    FROM public."Movie"').fetchall()

In [5]:
movies_df = pd.DataFrame(cursor, columns=["movie_id", "rating", "date", "runtime", "metascore", "imdb_rating", "boxoffice", "budget", "buzz"])
movies_df

,movie_id,rating,date,runtime,metascore,imdb_rating,boxoffice,budget,buzz
0,1,PG-13,2021-10-22,155,75.0,8.4,0,21000000,4
1,3,PG,2021-10-22,106,68.0,6.3,0,21000000,4
2,8,R,2021-10-29,106,98.0,8.0,0,21000000,4
3,12,R,2021-11-05,111,85.0,6.4,0,21000000,4
4,24,R,2021-11-19,110,69.0,6.2,0,21000000,4
5,30,R,2021-11-24,108,78.0,7.3,0,21000000,4
6,33,R,2021-12-03,128,78.0,7.0,0,21000000,4
7,39,R,2021-08-13,105,66.0,7.0,0,21000000,4


In [6]:
for index, row in movies_df.iterrows():
    # make sql call for people
    
    # make sql call for genres
    genre_query = engine.execute(f'SELECT public."Genre"."genre_name"\
        FROM public."Movie" \
        LEFT JOIN public."Movie_Genre_Junction" ON public."Movie"."movie_id" = public."Movie_Genre_Junction"."movie_id" \
        LEFT JOIN public."Genre" ON public."Movie_Genre_Junction"."genre_id" = public."Genre"."genre_id" \
        WHERE public."Movie"."movie_id" = {row["movie_id"]}').fetchall()
    for genre in genre_query:
        if genre[0] not in movies_df:
            movies_df[genre[0]] = 0
        movies_df.loc[movies_df["movie_id"] == row["movie_id"], [genre[0]]] = 1
    

In [7]:
movies_df

,movie_id,rating,date,runtime,metascore,imdb_rating,boxoffice,budget,buzz,Action,Adventure,Drama,Animation,Comedy,Biography,Romance
0,1,PG-13,2021-10-22,155,75.0,8.4,0,21000000,4,1,1,1,0,0,0,0
1,3,PG,2021-10-22,106,68.0,6.3,0,21000000,4,0,1,0,1,1,0,0
2,8,R,2021-10-29,106,98.0,8.0,0,21000000,4,0,0,1,0,0,0,0
3,12,R,2021-11-05,111,85.0,6.4,0,21000000,4,0,0,1,0,0,1,1
4,24,R,2021-11-19,110,69.0,6.2,0,21000000,4,0,0,1,0,0,0,0
5,30,R,2021-11-24,108,78.0,7.3,0,21000000,4,0,0,1,0,0,0,0
6,33,R,2021-12-03,128,78.0,7.0,0,21000000,4,0,0,1,0,1,0,0
7,39,R,2021-08-13,105,66.0,7.0,0,21000000,4,0,0,1,0,0,0,0


In [8]:
# save csv so we don't have to run the loop again
movies_df.to_csv("dummy_data_frame_upcoming.csv", index=False)

# Preprocess Data

In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

In [10]:
movies_df = pd.read_csv("dummy_data_frame_upcoming.csv")

In [11]:
movies_df

,movie_id,rating,date,runtime,metascore,imdb_rating,boxoffice,budget,buzz,Action,Adventure,Drama,Animation,Comedy,Biography,Romance
0,1,PG-13,2021-10-22,155,75.0,8.4,0,21000000,4,1,1,1,0,0,0,0
1,3,PG,2021-10-22,106,68.0,6.3,0,21000000,4,0,1,0,1,1,0,0
2,8,R,2021-10-29,106,98.0,8.0,0,21000000,4,0,0,1,0,0,0,0
3,12,R,2021-11-05,111,85.0,6.4,0,21000000,4,0,0,1,0,0,1,1
4,24,R,2021-11-19,110,69.0,6.2,0,21000000,4,0,0,1,0,0,0,0
5,30,R,2021-11-24,108,78.0,7.3,0,21000000,4,0,0,1,0,0,0,0
6,33,R,2021-12-03,128,78.0,7.0,0,21000000,4,0,0,1,0,1,0,0
7,39,R,2021-08-13,105,66.0,7.0,0,21000000,4,0,0,1,0,0,0,0


In [12]:

movies_df.drop(columns=["boxoffice"], inplace=True)

In [13]:
# clean data
# date into month
movies_df["month"] = pd.DatetimeIndex(movies_df['date']).month


In [14]:
# make dummies for rating
dummy_df = pd.get_dummies(movies_df, columns=["rating"])

In [15]:
dummy_df.drop(columns=["movie_id", "date"], inplace=True)
dummy_df

,runtime,metascore,imdb_rating,budget,buzz,Action,Adventure,Drama,Animation,Comedy,Biography,Romance,month,rating_PG,rating_PG-13,rating_R
0,155,75.0,8.4,21000000,4,1,1,1,0,0,0,0,10,0,1,0
1,106,68.0,6.3,21000000,4,0,1,0,1,1,0,0,10,1,0,0
2,106,98.0,8.0,21000000,4,0,0,1,0,0,0,0,10,0,0,1
3,111,85.0,6.4,21000000,4,0,0,1,0,0,1,1,11,0,0,1
4,110,69.0,6.2,21000000,4,0,0,1,0,0,0,0,11,0,0,1
5,108,78.0,7.3,21000000,4,0,0,1,0,0,0,0,11,0,0,1
6,128,78.0,7.0,21000000,4,0,0,1,0,1,0,0,12,0,0,1
7,105,66.0,7.0,21000000,4,0,0,1,0,0,0,0,8,0,0,1


In [16]:
load_df = pd.read_csv('dummy_data_frame_test.csv')

In [17]:
df = load_df.append(dummy_df)
df = df.fillna(0)
upcoming_movie_data = df.iloc[-8:]
#upcoming_movie_data.to_csv('upcoming_movie_data.csv', header=True, index=False)
upcoming_movie_data

,runtime,metascore,imdb_rating,budget,buzz,Adventure,Comedy,Family,Animation,Action,...,rating_G,rating_NC-17,rating_Not Rated,rating_PG,rating_PG-13,rating_R,rating_TV-14,rating_TV-MA,rating_TV-PG,rating_Unrated
0,155,75.0,8.4,21000000,4,1,0,0.0,0,1,...,0.0,0.0,0.0,0,1,0,0.0,0.0,0.0,0.0
1,106,68.0,6.3,21000000,4,1,1,0.0,1,0,...,0.0,0.0,0.0,1,0,0,0.0,0.0,0.0,0.0
2,106,98.0,8.0,21000000,4,0,0,0.0,0,0,...,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0
3,111,85.0,6.4,21000000,4,0,0,0.0,0,0,...,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0
4,110,69.0,6.2,21000000,4,0,0,0.0,0,0,...,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0
5,108,78.0,7.3,21000000,4,0,0,0.0,0,0,...,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0
6,128,78.0,7.0,21000000,4,0,1,0.0,0,0,...,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0
7,105,66.0,7.0,21000000,4,0,0,0.0,0,0,...,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0


In [18]:
dune = upcoming_movie_data[(upcoming_movie_data['metascore'] == 75.0)].values.tolist()
ron = upcoming_movie_data[(upcoming_movie_data['metascore'] == 68.0)].values.tolist()
souvenir = upcoming_movie_data[(upcoming_movie_data['metascore'] == 98.0)].values.tolist()
spencer = upcoming_movie_data[(upcoming_movie_data['metascore'] == 85.0)].values.tolist()
mothering_sunday = upcoming_movie_data[(upcoming_movie_data['metascore'] == 69.0)].values.tolist()
humans = upcoming_movie_data[(upcoming_movie_data['imdb_rating'] == 7.3)].values.tolist()
rocket	 = upcoming_movie_data[(upcoming_movie_data['metascore'] == 78.0) & (upcoming_movie_data['imdb_rating'] == 7.0)].values.tolist()
song = upcoming_movie_data[(upcoming_movie_data['metascore'] == 66.0)].values.tolist()
song

[[105.0,
  66.0,
  7.0,
  21000000.0,
  4.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  8.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0]]

In [19]:
model = pickle.load(open("rf_model.pkl", "rb"))

In [27]:
prediction = model.predict([[228,93,5.6,80400000,6,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,1,0,0,0,0,0,0]])
print(prediction)

['$121,000,000']
